In [226]:
import pandas as pd
from random import sample

In [227]:
drop_attribs = [
        "injury_status",
        "game_date",
        "opp",
        "spread",
        "over_under",
        "implied_team_score",
        "L5_dvp_rank",
        "L5_ppg_floor",
        "L5_ppg_avg",
        "L5_ppg_max",
        "Rest",
        "Opp Pace",
        "Opp DEff",
        "Opp DvP",
        "L2 FGA",
        "L5 FGA",
        "S FGA",
        "L2 Min",
        "L5 Min",
        "S Min",
        "L5 FP",
        "S FP",
        "Floor FP",
        "Ceil FP",
        'ppg_projection', 'value_projection', 'Proj FP', 'Proj Min', 'Proj Val',
        "position",
        "salary",  
]

In [228]:
def reduceForProjectionAnalysis(file):
    file['FantasyFuelPPGProj'] = file['ppg_projection']
    file['FantasyFuelValueProj'] = file['value_projection']
    file['DFN_PPGProj'] = file['Proj FP']
    file['DFN_MinProj'] = file['Proj Min']
    file['DFN_ValueProj'] = file['Proj Val']
    file.drop(drop_attribs, axis=1, inplace=True)
    file = file.loc[file['Actual FP'] > 0]
    return file



In [337]:
def verifyLineup(lineup):
    enough_represented_teams = len(set(lineup['team'].to_list()))>=2
    under_salary_cap = lineup['Salary'].sum() <= 50000
    all_unique_players = len(set(lineup['Player Name'].to_list())) == 8
    if enough_represented_teams and under_salary_cap and all_unique_players:
        return True
    else:
        return False
    
def createRandomPopulation(point_guards, shooting_guards, small_forwards, power_forwards, guards, forwards, centers, util, limit):
    counter = 0
    lineups = []
    while(counter<limit):
        lineup = pd.DataFrame()
        lineup = lineup.append(point_guards.sample(n=1))
        lineup = lineup.append(shooting_guards.sample(n=1))
        lineup = lineup.append(small_forwards.sample(n=1))
        lineup = lineup.append(power_forwards.sample(n=1))
        lineup = lineup.append(guards.sample(n=1))
        lineup = lineup.append(forwards.sample(n=1))
        lineup = lineup.append(centers.sample(n=1))
        lineup = lineup.append(util.sample(n=1))
        if(verifyLineup(lineup)):
            lineups.append(lineup)
            counter = counter + 1
    return lineups
        
    

In [356]:
def sortFitest(population): 
    population.sort(key=actualPointSum, reverse=True)
    return population 

def actualPointSum(lineup):
    return lineup['Actual FP'].sum() 

def evaluateFitness(population):
    sum = 0 
    for lineup in population[:10]:
        sum = sum + lineup['Actual FP'].sum()
    return sum/10 

def evaluateFitness2(population):
    sum = 0 
    for lineup in population[:20]:
        sum = sum + lineup['Actual FP'].sum()
    return sum/20


def performSelection(population):
    selected_population = population[:20]
    random_population = sample(population, 10)
    selected_population.extend(random_population)
    return selected_population

def performCrossover(population):
    counter = 0
    children = []
    while(counter < 15):
        parents = sample(population,2)
        children.append(mate(parents[0],parents[1]))
        counter = counter+1
    return children
        
def mate(lineup1, lineup2):
    point_guards = lineup1.loc[lineup1['Pos'].str.contains("PG")].append(lineup2.loc[lineup2['Pos'].str.contains("PG")])
    shooting_guards = lineup1.loc[lineup1['Pos'].str.contains("SG")].append(lineup2.loc[lineup2['Pos'].str.contains("SG")])
    small_forwards = lineup1.loc[lineup1['Pos'].str.contains("SF")].append(lineup2.loc[lineup2['Pos'].str.contains("SF")])
    power_forwards = lineup1.loc[lineup1['Pos'].str.contains("PF")].append(lineup2.loc[lineup2['Pos'].str.contains("PF")])
    guards = lineup1.loc[lineup1['Pos'].str.contains("G")].append(lineup2.loc[lineup2['Pos'].str.contains("G")])
    forwards = lineup1.loc[lineup1['Pos'].str.contains("F")].append(lineup2.loc[lineup2['Pos'].str.contains("F")])
    centers = lineup1.loc[lineup1['Pos'].str.contains("C")].append(lineup2.loc[lineup2['Pos'].str.contains("C")])
    util = lineup1.append(lineup2)
    child_lineup = lineup1.append(lineup2)
    while(not verifyLineup(child_lineup)):
        child_lineup = pd.DataFrame()
        child_lineup = child_lineup.append(point_guards.sample(n=1))
        child_lineup = child_lineup.append(shooting_guards.sample(n=1))
        child_lineup = child_lineup.append(small_forwards.sample(n=1))
        child_lineup = child_lineup.append(power_forwards.sample(n=1))
        child_lineup = child_lineup.append(guards.sample(n=1))
        child_lineup = child_lineup.append(forwards.sample(n=1))
        child_lineup = child_lineup.append(centers.sample(n=1))
        child_lineup = child_lineup.append(util.sample(n=1))
    return child_lineup
        
def createNextGeneration(old_population):
    selected_population = performSelection(old_population)
    children = performCrossover(selected_population)
    new_generation = old_population[:70]
    new_generation.extend(children)
    return new_generation
    
        

In [360]:
def geneticAlgorithmForHistorialBestLineups(file):
    point_guards = file.loc[file['Pos'].str.contains("PG")]
    shooting_guards = file.loc[file['Pos'].str.contains("SG")]
    small_forwards = file.loc[file['Pos'].str.contains("SF")]
    power_forwards = file.loc[file['Pos'].str.contains("PF")]
    guards = file.loc[file['Pos'].str.contains("G")]
    forwards = file.loc[file['Pos'].str.contains("F")]
    centers = file.loc[file['Pos'].str.contains("C")]
    util = file
    
    population = createRandomPopulation(point_guards, shooting_guards, 
                                               small_forwards, power_forwards, 
                                               guards, forwards, centers, util, 100)
    prevGenFitness = -100
    currentGenFitness = 0
    
    while(currentGenFitness-prevGenFitness>1):
        currentPopulation = sortFitest(population)
        prevGenFitness = currentGenFitness
        currentGenFitness = evaluateFitness2(currentPopulation)
        population = createNextGeneration(currentPopulation)
        #print(population[0]["Actual FP"].sum())
        
   
    return currentPopulation
    
    
        

In [361]:
test = reduceForProjectionAnalysis(pd.read_csv('HistoricalData_Merged/2020-01-26'))
pop = geneticAlgorithmForHistorialBestLineups(test)

,Player Name,team,Pos,Salary,Actual Min,Actual FP,Actual Val,FantasyFuelPPGProj,FantasyFuelValueProj,DFN_PPGProj,DFN_MinProj,DFN_ValueProj
12,Dennis Schroder,OKC,PG/SG,6800,40.6,43.3,6.4,36.5,5.37,35.5,36.0,5.2
6,Donovan Mitchell,UTA,PG/SG,7900,36.4,51.8,6.6,42.0,5.32,41.3,34.0,5.2
13,Andrew Wiggins,MIN,SG/SF,7100,42.8,63.8,9.0,35.0,4.93,36.8,34.5,5.2
41,P.J. Tucker,HOU,PF/C,4100,32.1,32.0,7.8,24.8,6.04,24.7,35.5,6.0
19,Buddy Hield,SAC,SG/SF,6600,37.7,56.3,8.5,32.0,4.85,27.8,25.5,4.2
26,Eric Gordon,HOU,SG/SF,6900,38.3,70.0,10.1,29.3,4.24,31.1,35.0,4.5
23,Jakob Poeltl,SA,C,3200,35.8,40.3,12.6,30.3,9.45,32.1,28.0,10.0
64,Danuel House Jr.,HOU,SF/PF,3900,45.6,47.8,12.3,19.0,4.87,24.2,33.0,6.2
